<a href="https://colab.research.google.com/github/RichardFrZ/FiltroAcoesViaveis/blob/main/src/Filtro_de_acoes_viaveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install investpy
import yfinance as yf
import pandas as pd
from datetime import datetime
from gspread_dataframe import set_with_dataframe as s
import pandas as pd
import gspread
from google.colab import auth
auth.authenticate_user()
import investpy

# Importa a lista de ações disponíveis no Brasil usando a biblioteca investpy.
lista = investpy.get_stocks(country='Brazil')

# Cria uma lista com os símbolos das ações.
symbol = lista['symbol'].tolist()

# Adiciona a extensão .SA nos ativos para a pesquisa.
tickers = symbol
tickers = [tickers + ".SA" for tickers in tickers]

# Define a data final como o ultimo dia do ano anterior.
data_final = datetime(datetime.now().year - 1, 12, 31) - pd.DateOffset(years= 0)

# Define a data inicial como 8 anos antes da data final.
data_inicial = data_final - pd.DateOffset(years= 8)

# Baixa os dados de dividendos.
dados = yf.download(tickers=tickers, start=data_inicial, end=data_final, actions=True)["Dividends"]

# Agrupa o DataFrame por ano e soma os dividendos para cada ano.
dados_agrupados = dados.groupby(dados.index.year).sum()

# Seleciona os últimos 5 anos de dados.
dados_agrupados = dados_agrupados.tail(5)

# Identifica colunas (empresas) que possuem valores de dividendos distribuídos iguais a zero.
df = dados_agrupados.columns[dados_agrupados.eq(0).any()]

# Remove essas empresas do DataFrame 'dados_agrupados'.
dados_agrupados = dados_agrupados.drop(columns=df)

# Calcula a média de dividendos para cada empresa restante.
dados_agrupados = dados_agrupados.mean()

# Calcula o preco teto dividindo o dividendo médio por 0.06 (6%) e arredondando para 2 casas decimais.
preco_teto = round(dados_agrupados/0.06,2)

#Busca os códigos dos ativos que foi possível calcular o preço teto.
tickers_selecionados = dados_agrupados.index.tolist()

#Busca os preços atuais dos ativos.
preco_atual = yf.download(tickers = tickers_selecionados, period='1mo')['Close']

#Seleciona apenas a última cotação.
preco_atual = round(preco_atual.iloc[-1],2)

#Busca a liquidez dos ativos em um período de 3 meses.
dados_volume = yf.download(tickers = tickers_selecionados, period='3mo')['Volume']

#Calcula a média de liquidez dos ativos.
dados_volume = round(dados_volume.mean())

#Seleciona apenas os ativos com liquidez maior que 1 Milhão.
liquidez = dados_volume[dados_volume > 1000000]

# Define o nome do índice como ATIVO.
for df in [preco_teto, preco_atual, liquidez]:
    df.index.name = "ATIVO"

# Criando um DataFrame chamado 'df_preco_teto'.
df_preco_atual = pd.DataFrame(preco_atual)

# Criando um DataFrame chamado 'df_liquidez'.
df_liquidez = pd.DataFrame(liquidez)

# Criando um DataFrame chamado 'df_preco_teto'.
df_preco_teto = pd.DataFrame(preco_teto)

# Define o nome da coluna como "PREÇO TETO".
df_preco_teto.columns = ["PREÇO TETO"]

# Define o nome da coluna como "PREÇO ATUAL".
df_preco_atual.columns = ["PREÇO ATUAL"]

# Define o nome da coluna como "LIQUIDEZ".
df_liquidez.columns = ["LIQUIDEZ"]

# Remove os últimos 3 caracteres (".SA") de cada ativo dos DataFrames 'df_preco_teto, df_preco_atual, df_liquidez'.
for df in [df_preco_teto, df_preco_atual, df_liquidez]:
    df.index = df.index.str[:-3]

# Combina os DataFrames Preço Teto e Preço Atual.
df_ativos = pd.merge(df_preco_teto, df_preco_atual, left_index=True, right_index=True)

# Combina os DataFrames df_ativos e Liquidez filtrando para apenas ativos com Liquidez.
df_ativos = pd.merge(df_ativos, df_liquidez, left_index=True, right_index=True)

# Filtra o DataFrame para ativos com preço atual menor que o preço teto.
df_ativos = df_ativos[df_ativos["PREÇO ATUAL"] < df_ativos["PREÇO TETO"]]

df_ativos

[*********************100%***********************]  747 of 747 completed
ERROR:yfinance:
169 Failed downloads:
ERROR:yfinance:['JCPC34.SA', 'XTED11.SA', 'SHDP11B.SA', 'BTTL3.SA', 'SULA11.SA', 'IGTA3.SA', 'BBRK3.SA', 'DAMT11B.SA', 'BIDI11.SA', 'CCXC3.SA', 'IDVL4.SA', 'VLJS11.SA', 'CARD3.SA', 'SPRI3.SA', 'DTEX3.SA', 'PCAR4.SA', 'SEDU3.SA', 'FAMB11B.SA', 'TIFF34.SA', 'DMMO3T.SA', 'RANI4.SA', 'NVIF11B.SA', 'HBTT11.SA', 'VVAR3.SA', 'JPSA3.SA', 'CESP3.SA', 'TOYB3.SA', 'SQIA3.SA', 'CEPE5.SA', 'PRTS11.SA', 'JBDU3.SA', 'MYPK12.SA', 'TOYB4.SA', 'ANCR11B.SA', 'IGBR3.SA', 'CELP5.SA', 'SFND11.SA', 'FVBI11.SA', 'LIQO3.SA', 'TESA3.SA', 'TASA17.SA', 'TCNO4.SA', 'BBFI11B.SA', 'BSEV3.SA', 'BMLC11B.SA', 'TRPN3.SA', 'CELP6.SA', 'CLGN34.SA', 'UTEC34.SA', 'PRSN11B.SA', 'IDVL9.SA', 'TIET2.SA', 'ELPL3.SA', 'MEND5.SA', 'ITEC3.SA', 'ELDO11B.SA', 'FINF11.SA', 'TIMP3.SA', 'LBRN34.SA', 'EDFO11B.SA', 'MTIG4.SA', 'SMLS3.SA', 'GESE11B.SA', 'LOGN12.SA', 'DOVL11B.SA', 'JRDM11.SA', 'PARD3.SA', 'CELP7.SA', 'TORM13.SA', '

,PREÇO TETO,PREÇO ATUAL,LIQUIDEZ
ATIVO,,,
BBDC4,18.32,15.81,35455439.0
BBSE3,40.57,36.73,4264312.0
BRAP4,66.32,18.59,2932980.0
BRSR6,16.32,12.58,1295312.0
CMIG4,12.85,11.89,12641229.0
CPFE3,40.37,34.55,1460833.0
CSMG3,25.97,24.88,1452747.0
CSNA3,22.81,11.61,6336750.0
GGBR4,23.76,18.60,10171736.0


### **Envio para o google sheets**

In [ ]:
# @title Necessário conectar conta Google

# Teste de saída de informações
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

cria_file = False

# Cria e define a planilha que irá a saída dos dados
if cria_file:
  gc.create('Ativos Viáveis')

worksheet = gc.open('Ativos Viáveis').worksheet('Página1')

# Limpa as colunas A até B na planilha especificada.
worksheet.batch_clear(["A:D"])

# Exporta o DataFrame 'df_preco_teto' para a planilha do Google Sheets, incluindo o índice.
s(worksheet, df_ativos, include_index= 'true')